Import necessary libraries

In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

Authenticate w/ Hugging Face Token

In [24]:
huggingface_token = userdata.get('TikTokTechJam2025')

if huggingface_token:
  print("HuggingFace token successfully found.")
else:
  print("HuggingFace token not found. Please add it to the Secrets manager using the name 'TikTokTechJam2025'.")

HuggingFace token successfully found.


Mount Google Drive & add project to path

In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/TikTok_Tech_Jam' 
sys.path.append(f'{project_path}/src')

print("Drive mounted and project path added.")

Import modules

In [ ]:
# Import .py files in src folder
from data_preprocessing import preprocess_data
from image_analysis import generate_image_descriptions_from_folder
from feature_engineering import get_unified_data
# from model_training import train_and_evaluate_model
# from policy_enforcement import classify_new_review

print("Modules imported successfully.")

Define file paths

In [ ]:
# Store/Organise all file paths in one place
RAW_DATA_PATH = f'{project_path}/data/reviews.csv'
PROCESSED_DATA_PATH = f'{project_path}/data/processed_reviews.csv'
IMAGE_FOLDER_PATH = f'{project_path}/data/image_dataset/'

MODEL_FEATURES_PATH = f'{project_path}/data/model_features.csv'
LOOKUP_TABLE_PATH = f'{project_path}/data/reviews_lookup.csv'
MODEL_SAVE_PATH = f'{project_path}/models/student_model.pkl'

print("File paths defined.")

Run the pipeline

In [ ]:
# === Step 1: Data Preprocessing ===
print("--- Running Data Preprocessing ---")
# Call the function from src/data_preprocessing.py
preprocess_data(
    input_file=RAW_DATA_PATH,
    output_file=PROCESSED_DATA_PATH,
    image_folder_path=IMAGE_FOLDER_PATH
)
print("--- Data Preprocessing Complete ---\n")

# === Step 2: Feature Engineering ===
print("--- Running Feature Engineering ---")
# Call the function from src/feature_engineering.py
get_unified_data(PROCESSED_DATA_PATH)
print("--- Feature Engineering Complete ---")

# === Step 3: Model Training ===
print("--- Running Model Training ---")
## (The next steps for model training will go in subsequent cells)

# === Step 4: Policy Enforcement ===
print("--- Running Policy Enforcement ---")
## (The next steps for policy enforcement will go in subsequent cells)